Análisis REM8
 ==========

Comienzo cargando todos los paquetes necesarios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime as dt
import datetime
from datetime import timedelta  
import timeboard as tb # turnos 

# para tratar de poner espanol
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time, format_timedelta, Locale
import locale                                    # para tratar de poner espanol
#locale = Locale('es', 'CL')
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')  

import os
import runpy
from pyexcel_ods import get_data
from matplotlib_venn import venn3
from matplotlib_venn import venn2, venn2_circles
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


#otro graficador
import seaborn as sns

#stats

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/home/egidio/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# cargo algunos scripts que me suelen servir para análisis

In [3]:
#listo los archivos
scripts = []
for dirname, dirnames, filenames in os.walk('/home/egidio/Dropbox/Compucosas/programas py'):
    for filename in filenames:
        scripts.append(os.path.join(dirname, filename))

scripts

# los cargo
for script in scripts:
    if script[-3:] == '.py':
        exec(open(str(script)).read())


['/home/egidio/Dropbox/Compucosas/programas py/lector_ODF.py',
 '/home/egidio/Dropbox/Compucosas/programas py/como importar',
 '/home/egidio/Dropbox/Compucosas/programas py/descripcion_univariada.py',
 '/home/egidio/Dropbox/Compucosas/programas py/características_operativas_de_test.py',
 '/home/egidio/Dropbox/Compucosas/programas py/percentiles.py',
 '/home/egidio/Dropbox/Compucosas/programas py/tabala_frecuencias.py',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/percentiles.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/lector_ODF.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/descripcion_univariada.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/características_operativas_de_test.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/tabala_frecuencias.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/funciones_locales.cpython-36.pyc']

# Cargo la BD

In [4]:
BD = pd.read_pickle('./BD/eSAMU.pkl')

In [5]:
BD.columns
BD.head(2)

Index(['Id', 'Fecha', 'Motivo del Llamado', 'Submotivo del Llamado',
       'Prioridad', 'Estado', 'Edad', 'Género', 'Dirección', 'Sector',
       'Comuna', 'Región', 'Intervención', 'Tipo Traslado',
       'Categoría Vehículo', 'Condición Traslado', 'En Regulación',
       'En Despacho', 'En Curso', 'Atendida', 'Completado', 'Regulado',
       'Usuario que Cierra', 'Móvil Agregado', 'Móvil Despachado',
       'Móvil Salida', 'Móvil Llegada Lugar', 'Móvil en Traslado',
       'Móvil Llegada a CA', 'Móvil Recepción Paciente',
       'Móvil Camilla Retenida', 'Móvil Vuelta a Base', 'Móvil LLegada a Base',
       'Móvil Finalizado', 'Nombre Vehículo', 'Tipo Requerido',
       'Tipo Despachado', 'Km Recorrido', 'Chofer', 'Paramédico', 'Enfermero',
       'Doctor', 'Longitud', 'Latitud', 'Diff_recep_Despachado',
       'Diff_Despacho_Salida', 'Diff_salida_enellugar', 'Diff_z6_1',
       'Diff_z6_2', 'Diff_z6_3'],
      dtype='object')

,Id,Fecha,Motivo del Llamado,Submotivo del Llamado,Prioridad,Estado,Edad,Género,Dirección,Sector,...,Enfermero,Doctor,Longitud,Latitud,Diff_recep_Despachado,Diff_Despacho_Salida,Diff_salida_enellugar,Diff_z6_1,Diff_z6_2,Diff_z6_3
0,62921,2018-12-31 23:46:09,Dolor o Problema específico,Problemas de Diabetes,S2,Completado,82.0,Masculino,calle providencia 078b,población los estanques,...,NaN,NaN,NaN,NaN,04:01:27,NaT,NaT,NaT,NaT,NaT
1,62920,2018-12-31 23:27:29,Dolor o Problema específico,"Embarazos, partos, abortos. Problemas puérperas",S2,Completado,27.0,Femenino,pasaje 3 1/2 casa 29,"c2 , reñaca alto",...,Karen Urra coloma,NaN,NaN,NaN,03:08:51,00:07:49,00:10:36,NaT,NaT,NaT


para efectos de este análisis es importante dividir ambulancias según su complejidad

In [6]:
BD['Bas_avan'] = BD['Tipo Despachado'] 
BD.loc[BD['Tipo Despachado'] == 'm1' , 'Bas_avan' ] = 'Básico'
BD.loc[BD['Tipo Despachado'] == 'm2' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'm3' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'x5' , 'Bas_avan' ] = 'Avanzado'

defino Bas_avan como categórico

# defino la BD con la que trabajaré

El informe a continuación es un análisis del:
* fecha
* sector

## según fecha

In [7]:
rango = (                                            #mes Diciembre
    (BD.Fecha > pd.to_datetime('2019-12-01')) &
    (BD.Fecha < pd.to_datetime('2019-12-31'))
)


db1 = BD[rango].copy()
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2019-12', 'M')], dtype=object)

## segun ambulancias

In [8]:
db1['Nombre Vehículo'].unique()

array([nan, 'R33', 'R20', 'R68', 'R100', 'R22', 'R23', 'R21', 'AP31',
       'R62', 'R31', 'R42', 'R61', 'AP 14', 'R24', 'R32', 'R84', 'R52',
       'AP2', 'W56', 'R11', 'R80', 'Móvil Artificio', 'AP41',
       'Móvil Nogales', 'R13', 'R71', 'R14', 'R10', 'AP28', 'R91', 'R25',
       'AP1', 'AP15', 'R57', 'Móvil Ventana', 'R53', 'AT90', 'R41',
       'AP16', 'AP26', 'AP51', 'R92', 'R72', 'nogales movil 8',
       'MOVIL PLACERES', 'W133 Carabineros', 'MOVIL RUTA', 'R51', 'R12',
       'R73', 'R43', 'R35', 'AP27', 'Móvil Hijuelas', 'sapu zapallar',
       'REMA Hospital Naval', 'Móvil Loncura', 'Móvil Puchuncaví', 'R81'],
      dtype=object)

elegir uno

In [10]:
#elegir uno

#sector = db1['Nombre Vehículo'].str.contains('R3') == True
#db1 = db1[sector]

In [11]:
texto1 = 'Este es un inforque que considera el mes de '
texto2 = 'En el territorio de '

sector = 'SSVQ'
#sector = db1.Comuna.value_counts().index[0]

mes= db1.Fecha.describe()[4].strftime('%B')

result = texto1 + mes + '. ' + texto2 + sector + '.' + '\n \n'
result
print(result, file=open('Informe.txt','a'))

'Este es un inforque que considera el mes de diciembre. En el territorio de SSVQ.\n \n'

In [12]:
db1.Comuna.value_counts()

Viña del Mar     1287
Quilpué           519
Quillota          383
Villa Alemana     377
Limache           236
Olmué             197
Quintero          175
La Ligua          144
Calera             98
Concón             73
Cabildo            47
Valparaíso         28
La Cruz            28
Hijuelas           20
Puchuncaví         18
Nogales            10
Petorca             7
Zapallar            6
Papudo              2
El Tabo             2
Putaendo            1
Name: Comuna, dtype: int64

# defino el archivo donde guardaré la info

Será un archivo

como dividir por fecha

In [13]:
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2019-12', 'M')], dtype=object)

## Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)

prestaciones según recurso

In [14]:
# los tiempos de z17 al z8 los divido según requisito planilla
cortado = pd.cut(
    db1.Diff_salida_enellugar.dt.seconds,
    [0, 20, 40, 1000000],
    labels = ['Menos 20 min',  '20 a 40 min',  'Más 40 min'])

# tabulo
#pd.crosstab(db1.Bas_avan, cortado, margins=True, margins_name= 'Total')

# tabulo por mes
print('Por Mes')
dum1 = cortado
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    result = pd.crosstab(
        test['Bas_avan'],
        dum1,
        margins=True, margins_name= 'Total', dropna=False)
    result.columns.name = 'Tiempo llegada al lugar'
    result.index.name = 'Tipo Móvil'
    result

Por Mes


Period('2019-12', 'M')

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,15,3,593,1166
Básico,12,6,839,1727
Total,34,9,1634,4714


escribo en el archivo

In [15]:
titulo = 'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'
texto = 'Se asume que todas las intervenciones críticas son realizadas \
por móviles avanzados y que las no-críticas por móviles básicos. Esto \
determina que las casillas de las prestaciones críticas tengan valores \
de cero para móviles básicos y las prestaciones no-críticas tengan \
valores cero para móviles avanzados. \n \n'
espacios = '\n \n'

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'

'Se asume que todas las intervenciones críticas son realizadas por móviles avanzados y que las no-críticas por móviles básicos. Esto determina que las casillas de las prestaciones críticas tengan valores de cero para móviles básicos y las prestaciones no-críticas tengan valores cero para móviles avanzados. \n \n'

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,15,3,593,1166
Básico,12,6,839,1727
Total,34,9,1634,4714


## Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA (Desde el lugar del evento a unidad de Emergencia)

In [16]:
db1['Categoría Vehículo'].value_counts()
#pd.crosstab(db1['Categoría Vehículo'], test['Bas_avan'])

db1['Categoría Vehículo'] = pd.Categorical(db1['Categoría Vehículo'],
                                           categories=['samu', 'enrutado', 'nosamu'])

samu        1232
enrutado     880
nosamu        18
Name: Categoría Vehículo, dtype: int64

In [17]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] != 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2019-12', 'M')

Categoría Vehículo  Tipo móvil
samu                Avanzado      358
                    Básico        448
enrutado            Avanzado      114
                    Básico        242
nosamu              Avanzado        3
                    Básico         13
Name: Id, dtype: int64

escribo en el archivo

In [18]:
titulo = ' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'

' '

Categoría Vehículo  Tipo móvil
samu                Avanzado      358
                    Básico        448
enrutado            Avanzado      114
                    Básico        242
nosamu              Avanzado        3
                    Básico         13
Name: Id, dtype: int64

## Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)

In [19]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] == 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2019-12', 'M')

Categoría Vehículo  Tipo móvil
samu                Avanzado      149
                    Básico        217
enrutado            Avanzado       44
                    Básico        242
Name: Id, dtype: int64

In [20]:
titulo = ' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

' '

Categoría Vehículo  Tipo móvil
samu                Avanzado      149
                    Básico        217
enrutado            Avanzado       44
                    Básico        242
Name: Id, dtype: int64

# Sección N: CLASIFICACION DE LAS INTERVENCIONES POR GRANDES GRUPOS DE DIAGNOSTICOS (SAMU)

In [21]:
db1['Patol'] = np.nan
db1.loc[db1['Submotivo del Llamado'] == 'Accidente vehicular o transporte. Atropellos',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Accidente múltiples víctimas (cualquier razón)',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Caídas graves',
        'Patol'] = 'PoliTMT'

db1.loc[db1['Submotivo del Llamado'] == 'Colapso respiratorio o circulatorio. PCR. Asfixia',
        'Patol'] = 'PCR'

db1.loc[db1['Submotivo del Llamado'] == 'Dolor de pecho',
        'Patol'] = 'SCA'

db1.loc[db1['Patol'].isnull(),
        'Patol'] = 'Otros'

db1['Patol'] = pd.Categorical(db1['Patol'], categories=['SCA', 'PCR', 'PoliTMT', 'Otros'])

In [22]:
db1.loc[db1['Género'] == 'Femenino', 'Género'] = 'Feme'
db1.loc[db1['Género'] == 'Masculino', 'Género'] = 'Masc'

db1['Género'] = pd.Categorical(db1['Género'], categories=['Masc', 'Feme'])

In [23]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    dum1 = pd.cut(test.Edad, range(0, 90, 5), right=False)  
    test['Patol'].value_counts(sort= False)
    result0 = pd.crosstab(test['Patol'], test['Género'])
    separador = ' -----------------------------------------------------------------'
    result1 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,:8]
    result2 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,8:16]
    result3 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,16:24]
    result4 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,24:]
    result0
    result1
    result2
    result3
    result4

Period('2019-12', 'M')

SCA         110
PCR          15
PoliTMT     803
Otros      3786
Name: Patol, dtype: int64

Género,Masc,Feme
Patol,,
SCA,47,57
PCR,8,6
PoliTMT,334,380
Otros,1721,1957


Edad    [0, 5)      [5, 10)      [10, 15)      [15, 20)     
Género    Masc Feme    Masc Feme     Masc Feme     Masc Feme
Patol                                                       
SCA          0    0       0    1        0    1        0    0
PCR          0    0       0    0        0    0        0    0
PoliTMT     15    7       5   16       15   12       10    7
Otros       56   42      25   22       50   34       49   69

Edad    [20, 25)      [25, 30)      [30, 35)      [35, 40)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            2    3        2    0        3    5        2    0
PCR            1    0        0    0        2    0        0    0
PoliTMT       24   27       51   36       54   35       19   26
Otros         88   75       71   75       82   76       61   69

Edad    [40, 45)      [45, 50)      [50, 55)      [55, 60)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            3    1        7    1        7    5        3    1
PCR            1    1        0    0        0    2        0    0
PoliTMT       16   22       20   10       29   32       13   62
Otros         62   54       62   82       93  102      107  107

Edad    [60, 65)      [65, 70)      [70, 75)      [75, 80)      [80, 85)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                                        
SCA            3    3        3    4        4    1        4    7        2    6
PCR            0    1        0    0        1    0        0    0        2    2
PoliTMT       16   24       11   12        8   21        6    3        5    8
Otros        132  113      116  123      143  168      167  162      127  191

In [24]:
titulo = ' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = 'Es una tabla larga, por lo que se divide en varios espacios '

titulo
texto
#result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result0, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result1, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result2, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result3, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result4, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(espacios, file=open('Informe.txt','a'))

' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

'Es una tabla larga, por lo que se divide en varios espacios '

# Finalización
Fin informe REM8

In [25]:
print('Fin informe REM 8', file=open('Informe.txt','a'))


# Análisis posteriores extra

In [26]:
db1[db1['Motivo del Llamado']=='Otro'][['Submotivo del Llamado', 'Id']]

,Submotivo del Llamado,Id
181,Fallecido,124395
221,Fallecido,124332
222,Fallecido,124331
238,Fallecido,124304
259,Fallecido,124262
262,Otros,124256
264,Fallecido,124254
271,Fallecido,124243
285,Fallecido,124221
310,Fallecido,124188
